# そもそもどんなデータを扱うの？

ここでは，本サイトで分析対象とするデータについて簡単に紹介し，基礎分析を実施します．

なお，このデータは[MADB Labでv1.0として公開されているもの](https://github.com/mediaarts-db/dataset/tree/1.0)に[こちらの前処理](https://kakeami.github.io/viz-madb/appendix/preprocess.html)を実施したものです．ご興味のある方はAppendixをご参照ください．

## 環境構築

In [66]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [67]:
# 前処理の結果，以下に分析対象ファイルが格納されていることを想定
PATH_DATA = '../../data/preprocess/out/magazines.csv'

In [68]:
df = pd.read_csv(PATH_DATA)

## 概要

`df`は

- 週刊少年ジャンプ
- 週刊少年サンデー
- 週刊少年マガジン
- 週刊少年チャンピオン

の`1970-08-02`から`2017-07-06`までのすべての掲載作品のデータを格納した`DataFrame`です．
まずはサイズを見てみましょう．

In [69]:
df.shape

(179866, 12)

各週の掲載作品を一行ずつ格納しているため，合計で約18万行程度の規模になります．
以下に，各列の構成を示します．

In [70]:
df.columns

Index(['mcname', 'miname', 'cname', 'epname', 'creator', 'pageStart',
       'pageEnd', 'numberOfPages', 'datePublished', 'price', 'publisher',
       'editor'],
      dtype='object')

- `mcname`: 雑誌名
- `miname`: 雑誌巻号名
- `cname`: マンガ作品名
- `epname`: 各話タイトル
- `creator`: 作者名
- `pageStart`: 開始ページ
- `pageEnd`: 終了ページ
- `numberOfPages`: 雑誌の合計ページ数
- `datePublished`: 雑誌の発行日
- `price`: 雑誌の価格
- `publisher`: 雑誌の発行元
- `editor`: 雑誌の編集者（編集長）

冒頭数行を見て，データのイメージを掴んでみましょう．

In [71]:
df.head(3).T

,0,1,2
mcname,週刊少年サンデー,週刊少年マガジン,週刊少年サンデー
miname,週刊少年サンデー 1970年 表示号数32,週刊少年マガジン 1970年 表示号数32,週刊少年サンデー 1970年 表示号数32
cname,ぶッかれ*ダン,アシュラ,男どアホウ甲子園
epname,NaN,NaN,NaN
creator,赤塚不二夫,ジョージ秋山,水島新司と水島プロ
pageStart,3.0,19.0,33.0
pageEnd,32.0,66.0,50.0
numberOfPages,288.0,282.0,288.0
datePublished,1970-08-02,1970-08-02,1970-08-02
price,80.0,80.0,80.0


`pandas`の`describe()`コマンドでざっくり集計してみます．

In [72]:
df.describe()

,pageStart,pageEnd,numberOfPages,price
count,179866.000000,179860.000000,179559.000000,179828.000000
mean,210.909330,228.364308,417.366002,203.870732
std,126.082707,122.080065,67.403584,41.928839
min,1.000000,0.000000,36.000000,80.000000
25%,107.000000,126.000000,356.000000,180.000000
50%,205.000000,222.000000,437.000000,210.000000
75%,305.000000,322.000000,464.000000,236.000000
max,9999.000000,1578.000000,600.000000,371.000000


- `pageStart`はすべての行でデータがありますが，それ以外は`NaN`（数値なし）が結構ありそうです
- `numberOfPages`の最小値，最大値が想定外に広がっていたので，あとで深堀りします
- `price`も同様です．後で深堀りします

次に，`NaN`の数を列ごとに集計します．

In [74]:
df.isna().sum()

mcname               0
miname               0
cname                9
epname           26814
creator            441
pageStart            0
pageEnd              6
numberOfPages      307
datePublished        0
price               38
publisher        24588
editor            2092
dtype: int64

```{margin} 前処理が原因の欠測
`numberOfPages`，`price`に関しては，想定外のパターンのデータを`NaN`に変換するよう[前処理](https://kakeami.github.io/viz-madb/appendix/preprocess.html#cm102)を施しています．
```

特に`epname`と`publisher`の欠損が多いことがわかります．

## 列ごとの基礎分析

### `mcname`（雑誌名）

In [75]:
df['mcname'].value_counts()

週刊少年サンデー      46153
週刊少年チャンピオン    45410
週刊少年マガジン      45293
週刊少年ジャンプ      43010
Name: mcname, dtype: int64

上記は`mcname`ごとの行数を表します．それぞれ同一期間で集計しましたが，掲載作品数の違いが生じています．

### `miname`（雑誌巻号名）

ユニークな`miname`数を集計します．

In [76]:
df['miname'].nunique()

9215

`mcname`（雑誌）ごとに集計した`miname`（雑誌巻号）は：

In [77]:
df.groupby('mcname')['miname'].nunique()

mcname
週刊少年サンデー      2303
週刊少年ジャンプ      2299
週刊少年チャンピオン    2313
週刊少年マガジン      2300
Name: miname, dtype: int64

ほぼ同数ですが，雑誌によって微妙に巻号数が異なることがわかります．

次は，`miname`（雑誌巻号）ごとに`mcname`（マンガ作品）数を集計してみます．

In [78]:
df.groupby('miname')['cname'].nunique().sort_values().reset_index()

,miname,cname
0,週刊少年マガジン 1975年 表示号数1,8
1,週刊少年マガジン 1971年 表示号数1,8
2,週刊少年マガジン 1974年 表示号数45,9
3,週刊少年サンデー 1973年 表示号数5,9
4,週刊少年サンデー 1974年 表示号数33,9
...,...,...
9210,週刊少年マガジン 2010年 表示号数7,35
9211,週刊少年ジャンプ 2014年 表示号数4,37
9212,週刊少年チャンピオン 2004年 表示号数1,37
9213,週刊少年ジャンプ 2003年 表示号数37,40


1970年付近の黎明期は掲載数が少なめだったようです．また，雑誌によってタイミングはバラバラですが，非常に沢山のマンガを掲載する**外れ値**が存在することもわかります．

### `cname`（マンガ作品名）

ユニークな`cname`（マンガ作品）数を集計します．

In [79]:
df['cname'].nunique()

6830

`mcname`（雑誌）ごとに`cname`（マンガ作品）数を集計します．

In [80]:
df.groupby('mcname')['cname'].nunique()

mcname
週刊少年サンデー      1485
週刊少年ジャンプ      2105
週刊少年チャンピオン    1713
週刊少年マガジン      1558
Name: cname, dtype: int64

ジャンプが圧倒的に多いですね…．試しに`cname`（マンガ作品）ごとに掲載数を集計してみます．

In [81]:
df_tmp = df[['mcname', 'cname']].value_counts().reset_index()
df_tmp.columns = ['mcname', 'cname', 'counts']
df_tmp

,mcname,cname,counts
0,週刊少年ジャンプ,こちら葛飾区亀有公園前派出所,1968
1,週刊少年マガジン,はじめの一歩,1184
2,週刊少年サンデー,名探偵コナン,1008
3,週刊少年ジャンプ,ONE PIECE,890
4,週刊少年サンデー,MAJOR,748
...,...,...,...
6856,週刊少年ジャンプ,大衆娯楽漫画きくらげ劇場,1
6857,週刊少年ジャンプ,大攻防戦,1
6858,週刊少年ジャンプ,大山岳,1
6859,週刊少年ジャンプ,大宮ジェット,1


```{margin} あれ，あの長編作品は…？
シーズンごとに作品名が変わっているシリーズ作品（ドカベン，刃牙，ジョジョ等）は，それぞれ別作品として集計されていることにご注意ください．
```

連載期間が長いものを見てみましょう．こち亀，はじめの一歩，名探偵コナン，ONEPIECE，MAJORと各雑誌のレジェンドが連なります．
一方で，連載期間が短いものの中には，企画ものや読み切りが存在するようです．

雑誌ごとに，マンガ作品の連載期間に関して基礎集計します．

In [82]:
df_tmp.groupby('mcname')['counts'].describe()

,count,mean,std,min,25%,50%,75%,max
mcname,,,,,,,,
週刊少年サンデー,1485.0,31.077441,73.995156,1.0,1.0,2.0,27.0,1008.0
週刊少年ジャンプ,2105.0,20.431829,75.075635,1.0,1.0,1.0,11.0,1968.0
週刊少年チャンピオン,1713.0,26.508465,60.133614,1.0,1.0,3.0,24.0,635.0
週刊少年マガジン,1558.0,29.068678,75.465729,1.0,1.0,2.0,21.0,1184.0


やはりジャンプの平均連載期間が，他誌と比べて短いことがわかります．

### `epname`（各話タイトル）

ユニークな`epname`（各話タイトル）数を集計します．

In [85]:
df['epname'].nunique()

150241

意外と重複しているようです．集計してみます．

In [87]:
df['epname'].value_counts().reset_index()

,index,epname
0,プロ野球編,186
1,後編,134
2,前編,118
3,最終回,82
4,第三部 孤狼青春編,81
...,...,...
150236,＄33 カミナリ、おそるべし,1
150237,星に願いを・・・・ ／ 笑っていいですよ ／ 朝食,1
150238,〔□67〕時間目 日替わり勉強!!,1
150239,C.C.S.9/ 亀裂,1


プロ野球編ってことは…．

In [88]:
df[df['epname']=='プロ野球編']['cname'].value_counts().reset_index()

,index,cname
0,ドカベン,186


やっぱりドカベンですね！ドカベンってもしかして**〇〇編**の粒度でしかタイトルをつけないのでしょうか…？ドカベンの`epname`を集計してみます．

In [92]:
df[df['cname']=='ドカベン']['epname'].value_counts().reset_index()

,index,epname
0,プロ野球編,186
1,新潟明訓対神奈川明訓,1
2,外伝 不知火・土門伝説,1
3,殿馬一人外伝 秘打「白鳥の湖」誕生秘話,1


In [93]:
df[df['cname']=='ドカベン']['epname'].isna().sum()

446

プロ野球編以外のドカベンの各話タイトルは欠測しているため，これ以上のことはわからなそうです．

### `creator`（作者）

`df`に存在する`creator`（作者）数を集計します．

In [96]:
df['creator'].nunique()

2740

合計作品数が多い`creator`を調べてみます．

In [98]:
df['creator'].value_counts().reset_index().head(10)

,index,creator
0,水島新司,2968
1,秋本治,1876
2,高橋留美子,1735
3,浜岡賢次,1381
4,青山剛昌,1285
5,あだち充,1270
6,森川ジョージ,1224
7,満田拓也,1222
8,板垣恵介,1171
9,藤田和日郎,1101


メンツが強すぎます…．
個人的にはこち亀の`秋本治`先生が一番かなと予想していましたが，`水島新司`先生が圧倒的でした．

ちなみに，こち亀の`creator`を集計すると以下のようになります．

In [103]:
df[df['cname']=='こちら葛飾区亀有公園前派出所']['creator'].value_counts().reset_index()

,index,creator
0,秋本治,1866
1,山止たつひこ,101
2,秋本治ととみさわちなつ,1


`秋本治`先生は，デビュー時`山止たつひこ`というペンネームを使っていました．

（この101話分を足しても全然追いつかない`水島新司`先生がすごすぎますが…）

### `pageStart`（開始ページ）

`pageStart`（開始ページ）について`describe()`で基礎集計すると，以下のようになります．

In [111]:
df['pageStart'].describe().reset_index()

,index,pageStart
0,count,179866.000000
1,mean,210.909330
2,std,126.082707
3,min,1.000000
4,25%,107.000000
5,50%,205.000000
6,75%,305.000000
7,max,9999.000000


異常に大きいものがあることがわかります．
試しに`pageStart`が`numberOfPage`より大きいものを抜き出すと：

In [110]:
df[df['pageStart'] > df['numberOfPages']].sort_values('pageStart')

,mcname,miname,cname,epname,creator,pageStart,pageEnd,numberOfPages,datePublished,price,publisher,editor
56163,週刊少年マガジン,週刊少年マガジン 1989年 表示号数10,コータローまかりとおる!,NaN,蛭田達也,49.0,67.0,36.0,1989-02-22,180.0,講談社 ∥ コウダンシャ,五十嵐隆夫
56166,週刊少年マガジン,週刊少年マガジン 1989年 表示号数10,SHOGUN,第39話 将軍!!緒方晋作!!,所十三,69.0,88.0,36.0,1989-02-22,180.0,講談社 ∥ コウダンシャ,五十嵐隆夫
56168,週刊少年マガジン,週刊少年マガジン 1989年 表示号数10,彼女はデリケート!,Lesson.7 コンピュータをたたきつぶせ,カジワラタケシ,89.0,108.0,36.0,1989-02-22,180.0,講談社 ∥ コウダンシャ,五十嵐隆夫
56169,週刊少年マガジン,週刊少年マガジン 1989年 表示号数10,THE STAR,NaN,島崎譲,109.0,128.0,36.0,1989-02-22,180.0,講談社 ∥ コウダンシャ,五十嵐隆夫
56171,週刊少年マガジン,週刊少年マガジン 1989年 表示号数10,キラキラ!,NaN,安達哲,129.0,148.0,36.0,1989-02-22,180.0,講談社 ∥ コウダンシャ,五十嵐隆夫
...,...,...,...,...,...,...,...,...,...,...,...,...
120378,週刊少年マガジン,週刊少年マガジン 2004年 表示号数38,予告大作戦,乙,西本英雄,563.0,563.0,551.0,2004-09-01,229.0,講談社 ∥ コウダンシャ,工富保
121886,週刊少年マガジン,週刊少年マガジン 2005年 表示号数2,もう、しませんから。,File.15 僕のジヒョンちゃんを紹介しますの巻,西本英雄,581.0,587.0,574.0,2005-01-12,238.0,講談社 ∥ コウダンシャ,工富保
171052,週刊少年サンデー,週刊少年サンデー 2015年 表示号数35,戦争劇場,第48戦:手を出したら最後…,土星フジコ,601.0,308.0,506.0,2015-07-29,270.0,NaN,市原武法
79256,週刊少年ジャンプ,週刊少年ジャンプ 1995年 表示号数1,ジョジョの奇妙な冒険,思い出したぞ!,荒木飛呂彦,2275.0,293.0,470.0,1995-01-01,200.0,集英社 ∥ シュウエイシャ,堀江信彦


となります．最後の2つは明らかに`startPage`がおかしい気がするので，分析をすすめる際は注意が必要そうです．

### `pageEnd`（終了ページ）

`pageEnd`（終了ページ）について`describe()`で基礎集計すると，以下のようになります．

In [113]:
df['pageEnd'].describe().reset_index()

,index,pageEnd
0,count,179860.000000
1,mean,228.364308
2,std,122.080065
3,min,0.000000
4,25%,126.000000
5,50%,222.000000
6,75%,322.000000
7,max,1578.000000


In [114]:
`pageEnd`にも異常に大きいものが存在します．

SyntaxError: invalid syntax (987282095.py, line 1)

### `numberOfPages`（各号の合計ページ数）

### `datePublished`（発行日）

### `publisher`（発行者）

### `editor`（編集者）